In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import re
from wordcloud import WordCloud, STOPWORDS
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import snscrape.modules.twitter as sntwitter
import nltk
import sys
from textblob import TextBlob


In [22]:
# Creating list to append tweet data to
tweet_list = []

keyword = (input("Please enter a keyword or phrase to focus your search: "))
NoOfTweets = int(input("Please enter the number of Tweets you would like to analyse: "))


# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper(f'{keyword} near:"United Kingdom" + since:2020-5-9 until:2020-5-10').get_items()):
    if i>=NoOfTweets:
        break
    tweet_list.append([tweet.content])
    

df = pd.DataFrame(tweet_list, columns=['Text'])

print(df)


Please enter a keyword or phrase to focus your search: lockdown 
Please enter the number of Tweets you would like to analyse: 2500
                                                   Text
0     @whitecarz @jasonplato But we have been prepar...
1     @johnredwood I’m taking a financial hit ,and c...
2     As much as I’ve hated every second of this loc...
3     Looks like we’re all buying puppies in lockdow...
4     said this a million times but I’m so lucky to ...
...                                                 ...
2495  @DementiaUK you are receiving many more calls ...
2496  Important updates to the men’s (16-46) scores,...
2497  This Bank holiday is such a weird one to celeb...
2498  #London #Lockdown diary #photography #streetph...
2499  People are taking the absolute piss with this ...

[2500 rows x 1 columns]


In [23]:
def cleanTxt(text):
    text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
    text = re.sub('#', '', text) # Removing '#' hash tags
    text = re.sub('RT[\s]+', '', text) # Removing RTs
    text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlinks
    
    return text

#applying this function to Text column of our dataframe
df["Text"] = df["Text"].apply(cleanTxt)

In [24]:
#Sentiment Analysis
def percentage(part,whole):
    return 100 * float(part)/float(whole)

#Assigning Initial Values
positive = 0
negative = 0
neutral = 0
polarity = 0
subjectivity = 0

#Creating empty lists
tweet_list1 = []
neutral_list = []
negative_list = []
positive_list = []

comp_sum = 0
comp_list = []
#Iterating over the tweets in the dataframe
for tweet in df['Text']:
    tweet_list1.append(tweet)
    #VADER
    analyzer = SentimentIntensityAnalyzer().polarity_scores(tweet)
    neg = analyzer['neg']
    neu = analyzer['neu']
    pos = analyzer['pos']
    comp = analyzer['compound']
    print(comp)
    
    comp_sum += comp
    comp_list.append(comp)
  
    #TextBlob
    analysis = TextBlob(tweet)
    subjectivity += analysis.sentiment.subjectivity
    polarity += analysis.sentiment.polarity
    
        
        
    if neg > pos:
        negative_list.append(tweet) #appending the tweet that satisfies this condition
        negative += 1 #increasing the count by 1
    elif pos > neg:
        positive_list.append(tweet) #appending the tweet that satisfies this condition
        positive += 1 #increasing the count by 1
    elif pos == neg:
        neutral_list.append(tweet) #appending the tweet that satisfies this condition
        neutral += 1 #increasing the count by 1 

        
    
positive_proportion = percentage(positive, NoOfTweets) #percentage is the function defined above
negative_proportion = percentage(negative, NoOfTweets)
neutral_proportion = percentage(neutral, NoOfTweets)
aggregate_subjectivity = subjectivity / NoOfTweets
aggregate_polarity = polarity / NoOfTweets
true_comp = comp_sum / NoOfTweets
valid_sum = positive_proportion + negative_proportion + neutral_proportion


0.2186
-0.1531
0.34
0.3612
0.9256
-0.5696
0.0
-0.7269
0.6369
0.0
0.0
0.0
0.25
0.296
0.0
0.4039
0.4215
0.5267
0.4184
-0.6219
-0.6447
-0.5106
0.9081
0.0
0.9902
0.9393
0.7269
-0.128
0.2235
0.0
-0.7691
-0.5093
-0.3182
0.2023
0.7506
-0.2624
-0.6779
0.0
-0.3612
0.8126
0.0
-0.5719
0.3147
-0.7603
0.0
0.3612
0.0
0.4767
-0.4215
-0.9248
-0.0772
-0.5719
0.0
-0.4767
0.3343
0.9274
0.5719
-0.8986
0.2263
-0.5207
0.9709
-0.128
-0.2411
-0.2484
0.0
-0.7744
0.0
0.0
0.0
0.1154
0.0
0.7506
-0.7184
0.5859
-0.69
0.6124
0.25
-0.6391
-0.8694
0.0
0.4939
0.5023
0.5267
-0.128
0.0
0.0
0.6369
-0.6597
0.9143
-0.2023
-0.6705
0.471
-0.5788
0.2348
0.1027
0.0
0.0
0.6249
0.2263
0.6106
0.474
-0.3164
0.0
0.1901
0.5903
0.7096
0.3612
0.0
-0.4767
0.7824
0.0
0.2263
-0.1531
-0.1779
0.0
0.2755
0.4087
0.5255
0.4019
0.296
0.6351
0.7772
0.0
0.836
0.7563
0.3204
-0.5719
0.0
-0.5574
-0.296
0.0
0.7783
0.0
-0.2023
0.4588
0.2584
0.7633
0.0
0.4404
0.0
-0.6115
0.7397
0.5575
0.3321
0.0
0.891
0.2975
0.1027
0.7724
0.4588
-0.3484
-0.6518
0.0
0.6

In [25]:
print(positive_proportion)
print(negative_proportion)
print(neutral_proportion)
print(valid_sum)
print(comp)
print(true_comp)

print(aggregate_polarity)
print(aggregate_subjectivity)
print(len(comp_list))

49.84
25.48
24.68
100.0
-0.2003
0.16757560000000016
0.10002362099786781
0.3973948729659231
2500
